In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc

from datetime import datetime
import requests
import json
from pandas import json_normalize

from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

In [2]:
today = datetime.today().strftime('%Y%m%d')
year_list =[2020, 2021, 2022]
df_ho =pd.DataFrame([])
for i in year_list:
    today_year = i

    key = 'UhnjyvmWEq5SH4yCqPylCPQSp4NdA524up%2FM1CrsUf3143Vmj3Rwx7PGHxq5YZY4mFHNHQ%2F0ixGwapRFApPsaw%3D%3D'
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear=' + str(today_year) + '&ServiceKey=' + str(key)
    response = requests.get(url)
    if response.status_code == 200:
        json_ob = json.loads(response.text)
        holidays_data = json_ob['response']['body']['items']['item']
        dataframe = json_normalize(holidays_data)
    df_ho = pd.concat([df_ho, dataframe])

In [20]:
df_ho=df_ho.reset_index()

In [21]:
df_ho['locdate']=df_ho['locdate'].reset_index(drop=True)

In [22]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [23]:
csv_to_parquet('C:/Users/gisic/gisic/jejutraffic/train.csv', 'train')
csv_to_parquet('C:/Users/gisic/gisic/jejutraffic/test.csv', 'test')

train Done.
test Done.


In [24]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [25]:
str_col = ['start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [71]:
t_train = train.copy()
t_test = test.copy()

In [72]:
y = t_train['target'] 

x = t_train.drop(['road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

t_test = t_test.drop(['road_name', 'start_node_name', 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code'], axis=1)

print(x.shape)
print(y.shape)
print(t_test.shape)

(4701217, 13)
(4701217,)
(291241, 12)


In [73]:
for i in df_ho['locdate']:
    x.loc[(x['base_date'] == i),'base_date'] = 1
x.loc[(x['base_date'] > 1),'base_date'] = 0

for i in df_ho['locdate']:
    t_test.loc[(t_test['base_date'] == i),'base_date'] = 1
    
t_test.loc[(t_test['base_date'] > 1),'base_date'] = 0    
x = x.replace('월', 0)
x = x.replace('화', 0)
x = x.replace('수', 0)
x = x.replace('목', 0)
x = x.replace('금', 0)
x = x.replace('토', 1)
x = x.replace('일', 1)
t_test = t_test.replace('월', 0)
t_test = t_test.replace('화', 0)
t_test = t_test.replace('수', 0)
t_test = t_test.replace('목', 0)
t_test = t_test.replace('금', 0)
t_test = t_test.replace('토', 1)
t_test = t_test.replace('일', 1)

In [74]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [77]:
## 훈련데이터세트의 shape ##
print(x.shape)
# (569,30) 

## 훈련세트, 테스트세트 shape ##
print(x_train.shape, x_test.shape)
# (455,30) (114,30)

## 훈련데이터세트의 클래스 비율 ##
np.unique(y, return_counts = True)
# (array([0,1]), array([212,357]))

## 훈련세트의 클래스 비율 ##
np.unique(y_train, return_counts = True)
# (array([0,1]), array([170,285]))

## 훈련세트의 클래스 비율 ##
np.unique(y_test, return_counts = True)
# (array([0,1]), array([42, 72]))

(4701217, 13)
(3760973, 13) (940244, 13)


(array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
        27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
        40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
        53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65.,
        66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77., 78.,
        79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90., 91.,
        92., 93., 94., 99.]),
 array([   13,    37,    67,   136,   262,   485,   766,  1026,  1433,
         2231,  2645,  3737,  4243,  5134,  6256,  7376,  8889, 10208,
        11071, 11840, 13013, 13406, 14214, 14862, 15329, 15951, 16375,
        17550, 18409, 19268, 19691, 20069, 20536, 18862, 18767, 18855,
        18113, 18710, 18326, 18524, 18595, 18463, 18582, 18707, 19559,
        20115, 21465, 22336, 22063, 21496, 21551, 21307, 21547, 21719,
        20974, 18471, 17351, 1523

##### str_list = ['start_latitude','start_longitude','end_latitude','end_longitude']
for i in str_list:
    if i == 'start_latitude' or i == 'end_latitude':
        X_train[i] -= 33
    else:
        X_train[i] -= 126
for i in str_list:
    if i == 'start_latitude' or i == 'end_latitude':
        test[i] -= 33
    else:
        test[i] -= 126

In [107]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', max_depth = 5)
xg_reg.fit(X_train, y_train)
pred = xg_reg.predict(test)


##### xg_reg = xgb.XGBRegressor(random_state=42).fit(X_train, y_train)

##### pred = xg_reg.predict(test)

In [108]:
sample_submission = pd.read_csv('C:/Users/gisic/gisic/jejutraffic/sample_submission.csv')

In [109]:
sample_submission['target'] = pred
sample_submission.to_csv("C:/Users/gisic/gisic/jejutraffic/submit.csv", index = False)

In [110]:
sample_submission

,id,target
0,TEST_000000,25.886963
1,TEST_000001,46.102360
2,TEST_000002,63.029503
3,TEST_000003,33.488358
4,TEST_000004,39.507397
...,...,...
291236,TEST_291236,52.457191
291237,TEST_291237,52.570896
291238,TEST_291238,22.827877
291239,TEST_291239,24.335571


In [94]:
suspect_submission = sample_submission.copy()

In [95]:
suspect_submission

,id,target
0,TEST_000000,24.408371
1,TEST_000001,45.870052
2,TEST_000002,63.367569
3,TEST_000003,34.505859
4,TEST_000004,37.616901
...,...,...
291236,TEST_291236,51.246002
291237,TEST_291237,52.160847
291238,TEST_291238,21.929148
291239,TEST_291239,25.345930
